# Lab:  Language model - Emojify 

In this assignment you will use word vector representations to build an Emojifier. 
🤩 💫 🔥

You'll implement a model which inputs a sentence (such as "Let's go see the baseball game tonight!") and finds the most appropriate emoji to be used with this sentence (⚾️).

1. You'll start with a baseline model (Emojifier-V1) using word embeddings.
2. Then you will build a more sophisticated model (Emojifier-V2) that further incorporates an LSTM. 

In [ ]:
# pip install emoji 

import numpy as np
from emo_utils import *
import emoji
import matplotlib.pyplot as plt
from test_utils import *

%matplotlib inline

## PART 1 - Baseline Model: Emojifier-V1

### 1.1 - Dataset EMOJISET

You have a tiny dataset (X, Y) where:
- X contains sentences(strings). At each row one sentence. 
- Y contains an integer label between 0 and 4 corresponding to an emoji for each sentence.

Load the dataset. It is split between training (*train_emoji.csv*) and testing (*test.csv*) data sets.

<img src="images/data_set.png" style="width:700px;height:300px;">
<caption><center><font color='purple'><b>Figure 1</b>: EMOJISET - a classification problem with 5 classes. A few examples of sentences. </center></caption>

In [ ]:
# function read_csv is part of emo_utils.py

X_train, Y_train = read_csv('data/train_emoji.csv')

X_test, Y_test = ?

#Check the dimension of each variable. 
?

# How many train examples (sentences) ? Print some of them.
?

# How many test examples (sentences) ? 
?


In [ ]:
# Guess what is maxLen ? 
maxLen = len(max(X_train, key=len).split())

#How many words consists the longest sentence? 
?


In [ ]:
# Print 1st sentence from X_train & corresponding label from Y_train. 
idx = 1
print(X_train[idx], label_to_emoji(Y_train[idx]))

#Make a cicle for to see the first 10 sentences. 

### 1.2 - Overview of the baseline model Emojifier-V1 

<center>
<img src="images/image_1.png" style="width:900px;height:300px;">
    <caption><center><font color='purple'><b>Figure 2</b>: Baseline model (Emojifier-V1).</center></caption>
</center></font>


#### Inputs and Outputs
* Input of the model is a string corresponding to a sentence (e.g. "I love you"). 
* Output is a vector of shape (1,5), indicating what is the probability of each emojis.
* The (1,5) probability vector is passed to an argmax layer, which extracts the index of the emoji with the highest probability.

#### One-hot Encoding
* To get the labels into a format suitable for training a softmax classifier, convert $Y$ from its current shape  $(m, 1)$ into a "one-hot representation" $(m, 5)$, 
    * Each row is a one-hot vector giving the label of one example.
    * `Y_oh` stands for "Y-one-hot" in the variable names `Y_oh_train` and `Y_oh_test`: 

In [ ]:
# Function convert_to_one_hot is part of emo_utils.py

Y_oh_train = convert_to_one_hot(Y_train, C = 5)

Y_oh_test = ?

See what `convert_to_one_hot()` did. Change `index` to print different values. 

In [ ]:
idx = 1

print(X_train[idx])

print(Y_train[idx])

print(label_to_emoji(Y_train[idx]))

print(Y_oh_train[idx])
      

<a name='1-3'></a>
### 1.3 - Implementing Emojifier-V1

As shown in Figure 2 (above), the first step is to:
* Convert each word in the input sentence into their word vector representations.
* Take an average of the word vectors. 

You will use pre-trained 50-dimensional GloVe embeddings. 

In [ ]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')

You've loaded:
- `word_to_index`: dictionary mapping from words to their indices in the vocabulary 
    - (400,001 words, with the valid indices ranging from 0 to 400,000)
- `index_to_word`: dictionary mapping from indices to their corresponding words in the vocabulary
- `word_to_vec_map`: dictionary mapping words to their GloVe vector representation. (50-dimensional)

In [ ]:
# What is the index of the word "cucumber" in the vocabulary ?
?

#What is the word in the vocabulary with index 289846 ?
?

###  Implement `sentence_to_avg()` 

Here, two steps are carried out:

1. Convert every sentence to lower-case, then split the sentence into a list of words. 
    * `X.lower()` and `X.split()` are useful.
2. For each word in the sentence, access its GloVe representation.
    * Then take the average of all of these word vectors.

In [ ]:
def sentence_to_avg(sentence, word_to_vec_map):
    """
    Converts a sentence into a list of words. 
    Extracts the GloVe representation of each word,
    averages its value into a single vector encoding the meaning 
    of the complete sentence.
    
    Arguments:
    sentence -- string, one training example from X
    word_to_vec_map -- dictionary mapping every word in the vocabulary 
    into its 50-dimensional vector representation
    
    Returns:
    avg -- average vector encoding information about the sentence, 
    numpy-array of shape (50,)
    """
    # Get just one word contained in the word_to_vec_map. 
    any_word = list(word_to_vec_map.keys())[0]
    
# STEP 1: Split sentence into list of lower case words
    words = sentence.lower().split()

    # Initialize the average word vector, 
    # should has the same shape as the word vectors.
    avg = np.zeros(word_to_vec_map[any_word].shape)
    
    # Initialize count to 0
    count = 0
    
# STEP 2: Average the word vectors. 
# Make loop for over all words in the list "words"
    for w in words:
        # Check that word exists in word_to_vec_map
        if w in list(word_to_vec_map.keys()):
            avg = avg + word_to_vec_map[w]
            # Increment count
            count +=1
          
    if count > 0:
        # Get the average, only if count > 0
        avg = ?
    
  
    return avg

### 1.4 Implement the baseline model (Fig.2)

After using `sentence_to_avg()` you need to:
* Pass the average through forward propagation
* Compute the cost
* Backpropagate to update the softmax parameters

* Equations to implement the forward pass & compute cross-entropy cost are below:
* Variable $Y_{oh}$ is one-hot encoding of output labels. 

$$ z^{(i)} = W . avg^{(i)} + b$$

$$ a^{(i)} = softmax(z^{(i)})$$

$$ \mathcal{L}^{(i)} = - \sum_{k = 0}^{n_y - 1} Y_{oh,k}^{(i)} * log(a^{(i)}_k)$$

In [ ]:
def model(X, Y,word_to_vec_map,learning_rate = 0.01,num_iterations=100):
    """
    Model to train word vector representations in numpy.
    
Arguments:
X -- input data, numpy array of sentences (strings) of shape (m,)
Y -- labels, numpy array of integers between 0 and 4 of shape (m,)
   
word_to_vec_map -- dictionary mapping every word in a vocabulary 
into its 50-dimensional vector representation
    
learning_rate --  for the stochastic gradient descent algorithm
num_iterations -- number of iterations 
    
Returns:
    pred -- vector of predictions, numpy-array of shape (m, 1)
    W -- weight matrix of the softmax layer, of shape (n_y, n_h)
    b -- bias of the softmax layer, of shape (n_y,)
    """
    
 # Get just one word contained in the word_to_vec_map. 
    any_word = ?
        
    # Initialize cost to 0
    cost = ?
    
    # Number of training examples ?
    m = ?     
    
    # Number of classes 
    n_y = len(np.unique(Y))    
    
    # dimensions of GloVe vectors 
    n_h = word_to_vec_map[any_word].shape[0] 
    
    # Initialize parameters using Xavier initialization
    W = np.random.randn(n_y, n_h) / np.sqrt(n_h)
    b = np.zeros((n_y,))
    
    # Convert Y to one hot encoding with n_y classes
    Y_oh = ?
    
    # Optimization loop
    # Loop over the number of iterations
    for t in range(num_iterations): 
        # Loop over the training examples
        for i in range(m):        

# Apply function sentence_to_avg to average the word vectors 
# from the i'th training example

            avg = ?

            # Forward propagate the avg through the softmax layer
            z = np.add(np.dot(W,avg),b)
            
            # function softmax() is part of emo_utils.py 
            a = softmax(z) 

# Compute cost using one hot representation of i'th training label
# and the output of the softmax (a)
            cost = -np.sum(np.dot(Y_oh[i], np.log(a)))
            
            # Compute gradients 
            dz = a - Y_oh[i]
            dW = np.dot(dz.reshape(n_y,1), avg.reshape(1, n_h))
            db = dz

            # Update parameters with Stochastic Gradient Descent
            # (after processing a single training examples)
            W = W - learning_rate * dW
            b = b - learning_rate * db
        
        if t % 10 == 0:
            print("Epoch: " + str(t) + " --- cost = " + str(cost))
            
            #Function predict is part of emo_utils.py
            pred = predict(X, Y, W, b, word_to_vec_map) 

    return pred, W, b

Train the model & learn softmax parameters (W, b). **Training may take some time !!!**

In [ ]:
np.random.seed(1)
pred, W, b = model(X_train, Y_train, word_to_vec_map)

### 1.5 Model Performance 


In [ ]:
# Apply function predict to get train & test accuracy of the trained model

# Training set: Accuracy arround 0.93 for 100 iterations"
pred_train = ?

# Test set: Accuracy arround 0.84
pred_test = ?

In [ ]:
def predict_single(sentence, W=W, b=b, word_to_vec_map=word_to_vec_map):
    """
    Given a sentence predict emojis.
    
    Arguments:
    sentence -- input data containing a sentence
    
    Returns:
    pred -- model predictions
    """

 # Get just one word contained in the word_to_vec_map. 
    any_word = ?
    
    # Dimension of Glove vectors   
    n_h = ?
        
    # Split jth test example (sentence) into list of lower case words
    words = ?

    #  # Inicialize average words' vectors to zeros
    avg = np.zeros((n_h,))
    count = 0
    for w in words:
        if w in word_to_vec_map:
            avg = avg + word_to_vec_map[w]
            #Increment count
            count = ?

    if count > 0:
        # Get the average, only if count > 0
        avg = ?

    # Forward propagation
    Z = np.dot(W, avg) + b
    A = softmax(Z)
    pred = np.argmax(A)
            
    return pred

In [ ]:
label_to_emoji(int(predict_single("I love you")))

#### The Model Matches Emojis to Relevant Words
In the training set, the algorithm saw the sentence 
>"I love you." with the label ❤️. 
* The word "adore" does not appear in the training set, let's see what happens with the sentence "I adore you."

In [ ]:
X_my_sentences = np.array(["i adore you", "i love you", "funny lol", 
"lets play with a ball", "food is ready", "not feeling happy", 
"This movie is not good and not enjoyable"])

Y_my_labels = np.array([[0], [0], [2], [1], [4],[3]])

pred = predict(X_my_sentences, Y_my_labels , W, b, word_to_vec_map)
print_predictions(X_my_sentences, pred)

Because *adore* has a similar embedding as *love*, the algorithm has generalized correctly even it has never seen before *adore*. 
Words such as *heart*, *dear*, *beloved* or *adore* have embedding vectors similar to *love*. 

**This algorithm ignores word ordering**, so it may not understand correctly phrases like "not happy" or "This movie is not good and not enjoyable". 

**Confusion Matrix** can help understand which classes are more difficult for the model. 

In [ ]:
# Print the confusion matrix for train and test data 
confusion_matrix(Y_test, pred_test)

<font color='blue'><b>What you should remember:</b>
- Even with only 132 training examples, you can get a reasonably good model for Emojifying. This is due to the generalization power of the word vectors. 
- Emojify-V1 will perform poorly on sentences such as "This movie is not good and not enjoyable" It doesn't understand combinations of words.
It just averages all the words' embedding vectors together, without considering the ordering of words. 
</font>
    
The next algorithm considers the ordering of words.

## PART 2 - Emojifier-V2: Using LSTMs

Here, LSTM model takes word **sequences** as input! This model is able to account for word ordering. 
Emojifier-V2 will continue to use pre-trained word embeddings to represent words. You'll feed word embeddings into an LSTM, and it will learn to predict the most appropriate emoji. 

In [ ]:
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.initializers import glorot_uniform


### 2.1 - Model Overview (Emojifier-v2)

<img src="images/emojifier-v2.png" style="width:700px;height:400px;"> <br>
<caption><center><font color='purple'><b>Figure 3</b>: Emojifier-V2. A 2-layer LSTM sequence classifier. </center></caption>

### 2.2  Padding Sequences of Varying Length

Here we want to train Keras using mini-batches. Most deep learning frameworks require that all sequences have the **same length**. Common solution to handle sequences of **different length** is to use padding.  Specifically:

    * Set a maximum sequence length
    * Pad all sequences to have the same length. 
    
#### Example of Padding:
* Given a maximum sequence length of 20, you could pad every sentence with "0"s so that each input sentence is of length 20. 
* Thus, the sentence "I love you" would be represented as $(e_{I}, e_{love}, e_{you}, \vec{0}, \vec{0}, \ldots, \vec{0})$. 
* In this example, any sentences longer than 20 words would have to be truncated. 
* One way to choose the max sequence length is to pick the length of the longest sentence in the training set. 

### 2.3  Embedding Layer

In Keras, the embedding matrix is represented as a "layer."

* The embedding matrix maps word indices to embedding vectors.
    * The word indices are positive integers.
    * The embedding vectors are dense vectors of fixed size.
    * A "dense" vector is the opposite of a sparse vector. It means that most of its values are non-zero.  As a counter-example, a one-hot encoded vector is not "dense."
* The embedding matrix can be derived in two ways:
    * Training a model to derive the embeddings from scratch. 
    * Using a pretrained embedding.
    
#### Using and Updating Pre-trained Embeddings 

[Embedding()](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) layer in Keras

* Initialize the Embedding layer with GloVe 50-dimensional vectors. 
* Keras allows you to either train or leave this layer fixed. Because the training set is small, you'll leave GloVe embeddings fixed instead of updating them.

#### Inputs and Outputs to the Embedding Layer

* The `Embedding()` layer's input is an integer matrix of size **(batch size, max input length)**. 
    * This input corresponds to sentences converted into lists of indices (integers).
    * The largest integer (the highest word index) in the input should be no larger than the vocabulary size.
* The embedding layer outputs an array of shape (batch size, max input length, dimension of word vectors).

* The figure shows the propagation of two example sentences through the embedding layer. 
    * Both examples have been zero-padded to a length of `max_len=5`.
    * The word embeddings are 50 units in length.
    * The final dimension of the representation is  `(2,max_len,50)`. 

<img src="images/embedding1.png" style="width:700px;height:250px;">
<caption><center><font color='purple'><b>Figure 4</b>: Embedding layer</center></caption>

#### Prepare the Input Sentences =>  Implement `sentences_to_indices`

This function processes an array of sentences X and returns inputs to the embedding layer:

* Convert each training sentences into a list of indices (the indices correspond to each word in the sentence)
* Zero-pad all these lists so that their length is the length of the longest sentence.

In [ ]:
for idx, val in enumerate(["I", "like", "learning"]):
    print(idx, val)

In [ ]:
def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices 
    corresponding to words in the sentences.
    The output shape should be such that it can be given to
    `Embedding()` 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- dictionary containing each word mapped to its index
    max_len -- maximum number of words in a sentence.
    Every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the 
    sentences from X, of shape (m, max_len)
    """
    
    # number of training examples
    m = ?   
    
# Initialize X_indices as a numpy matrix of zeros and the correct shape
    X_indices = ?
    
     # loop over training examples
    for i in range(m):        
        
# Convert ith training sentence in lower case and split is into words. 
# You should get a list of words.
        sentence_words = X[i].lower().split()
        
        # Initialize j 
        j = 0
        
        # Loop over the words of sentence_words

        for w in sentence_words:
            # if w exists in the word_to_index dictionary
            if w in word_to_index:
# Set the (i,j)th entry of X_indices to the index of the correct word.
                X_indices[i, j] = word_to_index[w]
                # Increment j
                j = ?

    
    return X_indices

In [ ]:
# Check what function `sentences_to_indices()` does for the 
# sentences in X1. Assume max number of words in a sentence =5

X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])

X1_indices = ?


#### Build Embedding Layer

Now you'll build the `Embedding()` layer in Keras, using pre-trained word vectors. 

* The embedding layer takes as input a list of word indices.
    * `sentences_to_indices()` creates these word indices.
* The embedding layer will return the word embeddings for a sentence. 

###  Implement `pretrained_embedding_layer()`:

1. Initialize the embedding matrix as a numpy array of zeros.
    * The embedding matrix has a row for each unique word in the vocabulary.
        * There is one additional row to handle "unknown" words.
        * So vocab_size is the number of unique words plus one.
    * Each row will store the vector representation of one word. 
        * For example, one row may be 50 positions long if using GloVe word vectors.
    * In the code below, `emb_dim` represents the length of a word embedding.
2. Fill in each row of the embedding matrix with the vector representation of a word
    * Each word in `word_to_index` is a string.
    * `word_to_vec_map` is a dictionary where the keys are strings and the values are the word vectors.
3. Define the Keras embedding layer. 
    * Use [Embedding()](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding). 
    * The input dimension is equal to the vocabulary length (number of unique words plus one).
    * The output dimension is equal to the number of positions in a word embedding.
    * Make this layer's embeddings fixed.
        * If you were to set `trainable = True`, then it will allow the optimization algorithm to modify the values of the word embeddings.
        * In this case, you don't want the model to modify the word embeddings.
4. Set the embedding weights to be equal to the embedding matrix.

In [ ]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained 
    GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector 
    representation.
    word_to_index -- dictionary mapping from words to their indices in
    the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer
    """
    # add 1 to to handle "unknown" words.
    vocab_size = len(word_to_index) + 1  
    
    # Get just one word contained in the word_to_vec_map. 
    any_word = ?
    
    # define dimensionality of GloVe word vectors (= 50)
     # dimensions of GloVe vectors 
    emb_dim = ?
      
    # Step 1
    # Initialize the embedding matrix as a numpy array of zeros
    # with shape [vocab_size,emb_dim]
    emb_matrix = ?
    
    # Step 2
    # Set each row "idx" of the embedding matrix to be 
# the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]

    # Step 3
    # Define Keras embedding layer with the correct input and output 
    #sizes. Make it non-trainable.
    embedding_layer = Embedding(vocab_size, emb_dim ,trainable = False)

    # Step 4
# Build the embedding layer, it is required before setting the weights 
# of the embedding layer. 
    embedding_layer.build((None,)) 
    
# Set the weights of the embedding layer to the embedding matrix. 
# The layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [ ]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][1] =", embedding_layer.get_weights()[0][1][1])
print("Input_dim", embedding_layer.input_dim)
print("Output_dim",embedding_layer.output_dim)

### 2.4 - Implement `Emojify_V2()`

Now you will build Emojifier-V2 model, in which you feed the embedding layer's output to an LSTM network! `Emojify_V2()`
function builds a Keras graph of the architecture shown in Fig.3.

* The model takes as input an array of sentences of shape (`m`, `max_len`, ) defined by `input_shape`. 
* The model outputs a softmax probability vector of shape (`m`, `C = 5`). 

* Keras layers:
    * [Input()](https://www.tensorflow.org/api_docs/python/tf/keras/Input)
        * Set the `shape` and `dtype` parameters.
        * The inputs are integers, so you can specify the data type as a string, 'int32'.
    * [LSTM()](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM)
        * Set the `units` and `return_sequences` parameters.
    * [Dropout()](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout)
        * Set the `rate` parameter.
    * [Dense()](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)
        * Set the `units`, 
    * [Activation()](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Activation)
        * You can pass in the activation of your choice as a lowercase string.
    * [Model()](https://www.tensorflow.org/api_docs/python/tf/keras/Model)
        * Set `inputs` and `outputs`.

* Here is some sample code: 
```Python
raw_inputs = Input(shape=(maxLen,), dtype='int32')
preprocessed_inputs = ... # some pre-processing
X = LSTM(units = ..., return_sequences= ...)(processed_inputs)
X = Dropout(rate = ..., )(X)
...
X = Dense(units = ...)(X)
X = Activation(...)(X)
model = Model(inputs=..., outputs=...)
...
```

In [ ]:
def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary 
    into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in 
    the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    # Define sentence_indices as the input of the graph.
    # It should be of shape input_shape and dtype 'int32' 
    # (as it contains indices, which are integers).
    sentence_indices = Input(shape=input_shape,dtype='int32')
    
    # Create the embedding layer pretrained with GloVe Vectors 
    embedding_layer = ?
    
    # Propagate sentence_indices through the embedding layer
     embeddings = embedding_layer(sentence_indices)   
    
# Propagate embeddings through LSTM layer with 128-dim. hidden state
# Returned output is a batch of sequences, set return_sequences = True
# If return_sequences = False, LSTM returns only last output in the sequence
    
    X = LSTM(units=128,return_sequences = True)(embeddings)
    
    # Add dropout with a probability of 0.5
    X = ?
    
# Propagate X trough another LSTM layer with 128-dim. hidden state
    
# Returned output should be single hidden state, not batch of sequences.
    X = LSTM(units=128,return_sequences = False)(X)
    
    # Add dropout with a probability of 0.5
    X = ?
    
    # Propagate X through a Dense layer with 5 units
    X = ?
    
    # Add a softmax activation
    X = ?
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices,outputs=X)
    
    return model

Run the following cell to create your model and check its summary. 

* Because all sentences in the dataset are less than 10 words, `max_len = 10` was chosen.  
* You should see that your architecture uses 20,223,927 parameters, of which 20,000,050 (the word embeddings) are non-trainable, with the remaining 223,877 being trainable. 
* Because your vocabulary size has 400,001 words (with valid indices from 0 to 400,000) there are 400,001\*50 = 20,000,050 non-trainable parameters. 

In [ ]:
model = ?
model.summary()

#### Compile the Model 

Use `categorical_crossentropy` loss, `adam` optimizer and `['accuracy']` metrics:

In [ ]:
?

### 2.5 - Train the Model 

Emojifier-V2 `model` takes as input an array of shape (`m`, `max_len`) and outputs probability vectors of shape (`m`, `number of classes`). 

Use function *sentences_to_indices* to convert X_train (array of sentences as strings) to X_train_indices (array of sentences as list of word indices). The same for X_test.

In [ ]:
X_train_indices = ?
X_test_indices = ?

Fit Keras model on `X_train_indices` & `Y_oh_train`, for `epochs = 50`, `batch_size = 32`, shuffle=True. 

In [ ]:
?

The model should perform around **90% to 100% accuracy** on the training set. Exact model accuracy may vary. 
Evaluate the model on test set: 

In [ ]:
loss, acc = model.evaluate(X_test_indices, Y_test_oh)

print("Test accuracy = ?")

You should get a test accuracy between 80% and 95%. Run the cell below to see the mislabelled examples: 

In [ ]:
C = 5
y_test_oh = np.eye(C)[Y_test.reshape(-1)]
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        print('Expected emoji:'+ label_to_emoji(Y_test[i]) + ' prediction: '+ X_test[i] + label_to_emoji(num).strip())

In [ ]:
#Test the model with the sentence "not feeling happy" and other examples

?


#### LSTM Version Accounts for Word Order
Emojify-V1 model was uncorrect for "not feeling happy", Emojify-V2 got it probably right! Emojify-V2 model still isn't very robust at understanding negation (such as "not happy").  This is because the training set is small and doesn't have a lot of examples of negation.

<font color='blue'><b>What you should remember</b>:
- For an NLP task where the training set is small, word embeddings may improve the algorithm. 
- Word embeddings allow the model to work on words in the test set that may not appear in the training set. 
- Training sequence models in Keras (and in most other deep learning frameworks) requires a few important details:
    - To use mini-batches, the sequences need to be **padded** so that all the examples in a mini-batch have the **same length**. 
    - An `Embedding()` layer can be initialized with pretrained values. 
        - These values can be either fixed or trained further on your dataset. 
        - If however your labeled dataset is small, it's usually not worth trying to train a large pre-trained set of embeddings.   
    - `Dropout()` right after `LSTM()` regularizes the network. 